## BRONZE TO SILVER LAYER


In [62]:
import pandas as pd
from common_utilities import global_path

### SILVER Layer - Process ProfitLoss History


In [63]:
# Read the CSV file into a DataFrame
df = pd.read_csv(global_path.tradehistory_gold_file_path)

# Convert 'datetime' column to datetime objects
df["datetime"] = pd.to_datetime(df["datetime"])

# Filter out rows where 'pnl_amount' is zero
df = df[df["pnl_amount"] != 0]

# Update 'side' column: 'SELL' becomes 'LONG', 'BUY' becomes 'SHORT'
df["side"] = df["side"].apply(lambda x: "LONG" if x == "SELL" else "SHORT" if x == "BUY" else x)

# Rename columns for clarity
df = df.rename(
    columns={
        "price": "close_side_price",
        "avg_price": "open_side_price",
    }
)

# Calculate open and close side amounts
df["open_side_amount"] = df["open_side_price"] * df["quantity"]
df["close_side_amount"] = df["close_side_price"] * df["quantity"]

# Calculate PnL percentage
df["pnl_percentage"] = ((df["close_side_amount"] - df["open_side_amount"]) / df["open_side_amount"]) * 100

# Sort the DataFrame by 'segment', 'stock_name', and 'datetime'
df = df.sort_values(by=["segment", "stock_name", "datetime"])

# Create a list of column names that contain the substrings 'price', 'amount', or 'pnl'
columns_to_round = [
    col  
    for col in df.columns
    if "price" in col or "amount" in col or "pnl" in col 
]

# Round the values in the selected columns to two decimal places
df[columns_to_round] = df[columns_to_round].round(2)

# Reset index to ensure it starts from 0
df = df.reset_index(drop=True)

# Select and reorder the columns for the final DataFrame
df = df[
    [
        "datetime",
        "exchange",
        "segment",
        "stock_name",
        "side",
        "quantity",
        "open_side_price",
        "open_side_amount",
        "close_side_price",
        "close_side_amount",
        "pnl_amount",
        "pnl_percentage"
    ]
]

# Save the final DataFrame to a CSV file
df.to_csv(global_path.profitloss_gold_file_path, index=None)

# Display the DataFrame information
df.info()

# Print the success message with the path of the saved file
print("GOLD Layer csv file for ProfitLoss successfully created at:")
print(global_path.profitloss_gold_file_path.resolve())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   datetime           83 non-null     datetime64[ns]
 1   exchange           83 non-null     object        
 2   segment            83 non-null     object        
 3   stock_name         83 non-null     object        
 4   side               83 non-null     object        
 5   quantity           83 non-null     float64       
 6   open_side_price    83 non-null     float64       
 7   open_side_amount   83 non-null     float64       
 8   close_side_price   83 non-null     float64       
 9   close_side_amount  83 non-null     float64       
 10  pnl_amount         83 non-null     float64       
 11  pnl_percentage     83 non-null     float64       
dtypes: datetime64[ns](1), float64(7), object(4)
memory usage: 7.9+ KB
GOLD Layer csv file for ProfitLoss successfully created 